<a href="https://colab.research.google.com/github/karjain/Anti_deepFake/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>